<a href="https://colab.research.google.com/github/hai105178362/blogs/blob/main/basic_ranking_2021_10_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: ranking

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/examples/basic_ranking"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/basic_ranking.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>



Real-world recommender systems are often composed of two stages:

1. The retrieval stage is responsible for selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.
2. The ranking stage takes the outputs of the retrieval model and fine-tunes them to select the best possible handful of recommendations. Its task is to narrow down the set of items the user may be interested in to a shortlist of likely candidates.

We're going to focus on the second stage, ranking. If you are interested in the retrieval stage, have a look at our [retrieval](basic_retrieval) tutorial.

In this tutorial, we're going to:

1. Get our data and split it into a training and test set.
2. Implement a ranking model.
3. Fit and evaluate it.


## Imports


Let's first get our imports out of the way.

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
import os
import pprint
import tempfile
import random

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're going to use the same data as the [retrieval](basic_retrieval) tutorial. This time, we're also going to keep the ratings: these are the objectives we are trying to predict.

In [ ]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "item_id": x["movie_title"],
    "item_name": x["movie_title"],
    "item_desc": x["movie_title"],
    "item_price": random.choice(range(9999, 13999)),
    "user_id": x["user_id"],
    "user_sex": random.choice(["male","female"]),
    "user_age": random.choice(range(1, 99)),
    "user_rating": x["user_rating"],
    "timestamp": x["timestamp"]
})

As before, we'll split the data by putting 80% of the ratings in the train set, and 20% in the test set.

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie titles present in the data. 

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [ ]:
item_ids = ratings.batch(1_000_000).map(lambda x: x["item_id"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_item_ids = np.unique(np.concatenate(list(item_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))
item_prices = np.concatenate(list(ratings.map(lambda x: x["item_price"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

item_descs= ratings.map(lambda x: x["item_desc"])

## Implementing a model

### Architecture

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures.

A model composed of multiple stacked dense layers is a relatively common architecture for ranking tasks. We can implement it as follows:

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    embedding_dimension = 32
    max_tokens  = 10000

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    self.sex_embeddings = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=["male","female"], mask_token=None),
        tf.keras.layers.Embedding(len(["male","female"]) + 1, embedding_dimension),
    ])

    #年龄分成几段
    self.age_embeddings = tf.keras.Sequential([
        tf.keras.layers.Discretization(bin_boundaries=[0., 3., 14., 18., 40., 55., 65.,130.]),
        tf.keras.layers.Embedding(len([0., 3., 14., 18., 40., 55., 65.,130.]) + 1, embedding_dimension),
    ])

    self.timestamp_embedding = tf.keras.Sequential([
        tf.keras.layers.Discretization(timestamp_buckets.tolist()),
        tf.keras.layers.Embedding(len(timestamp_buckets) + 1, embedding_dimension),
    ])

    # self.normalized_timestamp = tf.keras.layers.Normalization(
    #     axis=None
    # )

    # self.normalized_timestamp.adapt(timestamps)

    # Compute embeddings for movies.
    self.item_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_item_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension)
    ])

    self.desc_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.desc_text_embedding = tf.keras.Sequential([
      self.desc_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.desc_vectorizer.adapt(item_descs)

    self.normalized_price = tf.keras.layers.Normalization(
        axis=None
    )

    self.normalized_price.adapt(item_prices)

  #   # Compute predictions.
  #   self.ratings = tf.keras.Sequential([
  #     # Learn multiple dense layers.
  #     tf.keras.layers.Dense(256, activation="relu"),
  #     tf.keras.layers.Dense(64, activation="relu"),
  #     # Make rating predictions in the final layer.
  #     tf.keras.layers.Dense(1)
  # ])
    # Then construct the layers.
    self.ratings = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes:
      self.ratings.add(tf.keras.layers.Dense(layer_size, activation="relu"))
    # No activation for the last layer.
    self.ratings.add(tf.keras.layers.Dense(1))  
  def call(self, inputs):

    user_id, user_sex,user_age, item_id,item_desc, item_price, timestamp = inputs

    user_embedding = self.user_embeddings(user_id)
    sex_embedding = self.sex_embeddings(user_sex)
    age_embedding = self.age_embeddings(user_age)
    item_embedding = self.item_embeddings(item_id)
    desc_text_embedding = self.desc_text_embedding(item_desc)
    price_embedding = tf.reshape(self.normalized_price(item_price), (-1, 1))
    timestamp_embedding = self.timestamp_embedding(timestamp)

    return self.ratings(tf.concat([user_embedding, sex_embedding, age_embedding, item_embedding,desc_text_embedding, timestamp_embedding], axis=1))

This model takes user ids and movie titles, and outputs a predicted rating:

In [ ]:
RankingModel([256,64])((
    np.array(["42"]),
    np.array(["female"]),
    np.array([22]),
    np.array(["One Flew Over the Cuckoo's Nest (1975)"]),
    np.array(["One Flew Over the Cuckoo's Nest (1975)"]), 
    np.array([9999]),
    np.array([1198090049])
     ))

### Loss and metrics

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation. 

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [ ]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

The task itself is a Keras layer that takes true and predicted as arguments, and returns the computed loss. We'll use that to implement the model's training loop.

### The full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines bulding models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(layer_sizes)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )


  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    #labels = features.pop("user_rating")
    
    rating_predictions = self.ranking_model(
        (features["user_id"],features["user_sex"],features["user_age"], features["item_id"],features["item_desc"],features["item_price"], features["timestamp"]))

    # The task computes the loss and the metrics.
    return self.task(labels=features["user_rating"], predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [ ]:
model = MovielensModel([256,64])
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [ ]:
model.fit(cached_train, epochs=3)

As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [ ]:
model.evaluate(cached_test, return_dict=True)

The lower the RMSE metric, the more accurate our model is at predicting ratings.

## Testing the ranking model

Now we can test the ranking model by computing predictions for a set of movies and then rank these movies based on the predictions:


In [ ]:
class ModelAbc(tf.keras.Model):
    def __init__(self, rank_model : tf.keras.Model):
      super().__init__()
      self.rank_model = rank_model
    def call(self,features: Dict[str, tf.Tensor]):
     return self.rank_model.ranking_model(
        (features["user_id"],features["user_sex"],features["user_age"], features["item_id"],features["item_desc"],features["item_price"], features["timestamp"]))

In [ ]:
modelAbc = ModelAbc(model)
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = modelAbc({
      "user_id": np.array(["42"]),
      "item_id": np.array([movie_title]),
      "item_desc": np.array([movie_title]),
      "user_sex": np.array(["female"]),
      "user_age": np.array([22]),
      "item_price": np.array([10999]),
      "timestamp": np.array([1198090049])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")


## Exporting for serving

The model can be easily exported for serving:


In [ ]:
path = os.path.join("/content/sample_data", "model")
tf.saved_model.save(modelAbc, path)

We can now load it back and perform predictions:

In [ ]:
loaded = tf.saved_model.load(path)

loaded({
      "user_id": np.array(["42"]),
      "item_id": [movie_title],
      "item_desc": [movie_title],
      "user_sex": np.array(["female"]),
      "user_age": np.array([22]),
      "item_price": np.array([10999]),
      "timestamp": np.array([1198090049])
  }).numpy()

## Next steps

The model above gives us a decent start towards building a ranking system.

Of course, making a practical ranking system requires much more effort.

In most cases, a ranking model can be substantially improved by using more features rather than just user and candidate identifiers. To see how to do that, have a look at the [side features](featurization) tutorial.

A careful understanding of the objectives worth optimizing is also necessary. To get started on building a recommender that optimizes multiple objectives, have a look at our [multitask](multitask) tutorial.